In [ ]:
import os
import requests
from bs4 import BeautifulSoup as bs

In [110]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta, date
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import time 
import pandas as pd
from io import StringIO

In [3]:
url = "https://www.tankauction.com/"
payload = 
header= {'client_id':'yjj7479', 'password':'xodzmdhrtus0911!'}
r = requests.post(url,data = payload)
print(r.url, r.status_code)
print(r.text)

SyntaxError: invalid syntax (2714571026.py, line 2)

In [13]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )


driver = webdriver.Chrome()

driver.get('https://www.tankauction.com/')

wait = WebDriverWait(driver, 10)

# 로그인 버튼 선택
button = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.hand')
    ))
button.click()

driver.find_element(By.CSS_SELECTOR, '.hand').send_keys('yjj7479')



# 로그인 폼 채우기
driver.find_element(By.ID, 'client_id').send_keys('yjj7479')
driver.find_element(By.ID, 'password').send_keys('xodzmdhrtus0911!')
driver.find_element(By.ID, "button btn_white bigrounded").click()

time.sleep(5)  # 로그인 처리 대기

# 보호된 페이지 접근
driver.get('https://www.tankauction.com/inc/BgItems.php?title=%EC%95%84%ED%8C%8C%ED%8A%B8&t=a1&tid=2276168&dong=+52+&tg=A&tp=1&a=1&b=1&c=1&pnu=1132010500&addr=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EB%8F%84%EB%B4%89%EA%B5%AC+%EC%8C%8D%EB%AC%B8%EB%8F%99&bg=59-3&bldgNm=%ED%95%9C%EC%96%915&bldgSqm=65.37&bldgNmCnt=1&bldgDj=%ED%95%9C%EC%96%915&roadAddr=%ED%95%B4%EB%93%B1%EB%A1%9C25%EA%B8%B8+36&gb=1&tile_mode=&area=&search_mode=&class=&fromYear=2006&toYear=2025&fromMonth=1&toMonth=4&dealType=1&houseType=&landuse=&zoning=&oflanduse=&buildinguse=&area_gp=&pageNo=1&dataSize=20&pageSize=10&BBL=1&SBL=1&DBL=1')
html = driver.page_source

print(html)

driver.quit()

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [91]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

지역을 입력해주세요 서울
주거용과 아파트 중 하나를 입력해주세요 아파트


In [111]:

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )

driver.get('https://www.tankauction.com/')

wait = WebDriverWait(driver, 10)
# 로그인
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.hand'))).click()
wait.until(EC.element_to_be_clickable((By.ID, 'client_id'))).send_keys('yjj7479')
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# 경매 검색
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()

Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# 날짜 선택
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# 면적 설정
bldgSqmEnd_input = wait.until(EC.element_to_be_clickable((By.ID, "bldgSqmEnd")))
bldgSqmEnd_input.clear()
bldgSqmEnd_input.send_keys(85)

# 검색 버튼 클릭
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()

# 리스트 갯수 조정
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')

# 리스트 항목 로딩
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
# 렌더링이 완전히 끝났는지 확인 (JavaScript DOM 상태 체크)
WebDriverWait(driver, 10).until(
    lambda d: d.execute_script("return document.readyState") == "complete"
)


print("🔍 리스트 로딩 완료")

# row_id 수집
row_ids = []
for retry in range(3):
    try:
        rows = driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand')
        for row in rows:
            row_ids.append(row.get_attribute("id"))
        break
    except StaleElementReferenceException:
        print("⚠️ StaleElement 발생, row 재시도 중...")
        time.sleep(1)

# 각 row에서 데이터 추출
result1 = []
for row_id in row_ids:
    try:
        item = driver.find_element(By.ID, row_id)
        tid = item.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
        num = item.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
        address = item.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}').get_attribute("innerText")
        start_price = int(item.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))
        end_price = int(item.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))
        the_date = item.find_element(By.ID, f"bidDt_{tid}").text

        view_td = item.find_elements(By.CSS_SELECTOR, 'td')[-1]
        view_divs = view_td.find_elements(By.TAG_NAME, 'div')
        pop = view_divs[1].text.split("\n")[0] if len(view_divs) > 1 else None

        result1.append([num, address, start_price, end_price, the_date, pop])
        print(num, address, start_price, end_price, the_date, pop)

        # 팝업 열기 및 닫기
        main_window = driver.current_window_handle
        clickable = item.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
        driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", clickable)
        
        WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
        new_window = [w for w in driver.window_handles if w != main_window][0]
        driver.switch_to.window(new_window)
        # 팝업 데이터 추출 위치
        driver.close()
        driver.switch_to.window(main_window)

    except Exception as e:
        print(f"❗ 오류 발생 (ID: {row_id}):", str(e).splitlines()[0])

    

🔍 리스트 로딩 완료
⚠️ StaleElement 발생, row 재시도 중...
2024-52691 서울 강동구 길동 387-5, 7층709호 (길동,길동청광플러스원큐브1차) 외 1필지 120000000 120000000 25.04.28 
2024-54246 서울 송파구 가락동 78-1, 15층1507호 (가락동,한화오벨리스크) 189000000 189000000 25.04.28 
2023-4493(2) 서울 동대문구 이문동 327-7, 14층1412호 (이문동,세진빌딩) 외 6필지 329000000 263200000 25.04.29 
2023-125267 서울 영등포구 영등포동1가 145, 제9층 제913호 134000000 43909000 25.04.29 
2024-3282 서울 도봉구 창동 811, 101동 2층205호 (도봉동,대동아파트) 485000000 388000000 25.04.29 
2024-106495 서울 도봉구 도봉동 641, 114동 11층1105호 (도봉동,서원아파트) 외 1필지 399000000 319200000 25.04.29 
2024-109326 서울 동대문구 전농동 6, 16동 1층117호 (전농동,전농우성아파트) 외 1필지 536000000 343040000 25.04.29 
2024-111985 서울 도봉구 방학동 272, 16동 12층1210호 (방학동,신동아아파트) 374000000 299200000 25.04.29 
2024-112223 서울 노원구 상계동 639, 103동 12층1213호 (상계동,보람아파트) 외 1필지 411000000 328800000 25.04.29 
2024-114462 서울 강서구 방화동 832, 제303동 제4층 제408호 (방화동, 방화아파트) 189000000 151200000 25.04.29 
2024-142900 서울 강서구 화곡동 938-23, 제8층 제802호 (화곡동, 에스엠케이아파트) 294000000 294000000 25.04.30 
2023-107195 서울 관악구 신림

In [108]:
result1 = []
for row_id in row_ids:
    try:
        item = driver.find_element(By.ID, row_id)
        tid = item.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
        num = item.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
        address = item.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}').get_attribute("innerText")
        start_price = int(item.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))
        end_price = int(item.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))
        the_date = item.find_element(By.ID, f"bidDt_{tid}").text

        view_td = item.find_elements(By.CSS_SELECTOR, 'td')[-1]
        view_divs = view_td.find_elements(By.TAG_NAME, 'div')
        pop = view_divs[1].text.split("\n")[0] if len(view_divs) > 1 else None

        result1.append([num, address, start_price, end_price, the_date, pop])
        print(num, address, start_price, end_price, the_date, pop)

        # 팝업 열기 및 닫기
        main_window = driver.current_window_handle
        clickable = item.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
        driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", clickable)
        
        WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
        new_window = [w for w in driver.window_handles if w != main_window][0]
        driver.switch_to.window(new_window)
        # 팝업 데이터 추출 위치
        driver.close()
        driver.switch_to.window(main_window)

    except Exception as e:
        print(f"❗ 오류 발생 (ID: {row_id}):", str(e).splitlines()[0])

    

2024-52691 서울 강동구 길동 387-5, 7층709호 (길동,길동청광플러스원큐브1차) 외 1필지 120000000 120000000 25.04.28 
2024-54246 서울 송파구 가락동 78-1, 15층1507호 (가락동,한화오벨리스크) 189000000 189000000 25.04.28 
2023-4493(2) 서울 동대문구 이문동 327-7, 14층1412호 (이문동,세진빌딩) 외 6필지 329000000 263200000 25.04.29 
2023-125267 서울 영등포구 영등포동1가 145, 제9층 제913호 134000000 43909000 25.04.29 
2024-3282 서울 도봉구 창동 811, 101동 2층205호 (도봉동,대동아파트) 485000000 388000000 25.04.29 
2024-106495 서울 도봉구 도봉동 641, 114동 11층1105호 (도봉동,서원아파트) 외 1필지 399000000 319200000 25.04.29 
2024-109326 서울 동대문구 전농동 6, 16동 1층117호 (전농동,전농우성아파트) 외 1필지 536000000 343040000 25.04.29 
2024-111985 서울 도봉구 방학동 272, 16동 12층1210호 (방학동,신동아아파트) 374000000 299200000 25.04.29 
2024-112223 서울 노원구 상계동 639, 103동 12층1213호 (상계동,보람아파트) 외 1필지 411000000 328800000 25.04.29 
2024-114462 서울 강서구 방화동 832, 제303동 제4층 제408호 (방화동, 방화아파트) 189000000 151200000 25.04.29 
2024-142900 서울 강서구 화곡동 938-23, 제8층 제802호 (화곡동, 에스엠케이아파트) 294000000 294000000 25.04.30 
2023-107195 서울 관악구 신림동 1421-1, 10층1004호 (신림동,신림큐브) 140000000 15032

In [20]:


# 로그인 폼 채우기
driver.find_element(By.ID, 'client_id').send_keys('yjj7479')
time.sleep(2)
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()

time.sleep(5)  # 로그인 처리 대기


In [112]:
# 팝업 열기 및 닫기
main_window = driver.current_window_handle
clickable = item.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
time.sleep(1)
driver.execute_script("arguments[0].click();", clickable)

WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
new_window = [driver.window_handles][0]
driver.switch_to.window(new_window)
# 팝업 데이터 추출 위치
driver.close()
driver.switch_to.window(main_window)


InvalidArgumentException: Message: invalid argument: 'handle' must be a string
  (Session info: chrome=125.0.6422.61)
Stacktrace:
0   chromedriver                        0x000000010fc096b8 chromedriver + 6149816
1   chromedriver                        0x000000010fc00cda chromedriver + 6114522
2   chromedriver                        0x000000010f68dd5c chromedriver + 400732
3   chromedriver                        0x000000010f70e80a chromedriver + 927754
4   chromedriver                        0x000000010f6fdc78 chromedriver + 859256
5   chromedriver                        0x000000010f6cd6d5 chromedriver + 661205
6   chromedriver                        0x000000010f6cdf6e chromedriver + 663406
7   chromedriver                        0x000000010fbcacf0 chromedriver + 5893360
8   chromedriver                        0x000000010fbd04bc chromedriver + 5915836
9   chromedriver                        0x000000010fbac8b4 chromedriver + 5769396
10  chromedriver                        0x000000010fbd0

In [34]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

# 경매 검색 
btn_search = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, ".menuicon_10")
    ))
btn_search.click()
# 지역 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd"))))
siCd_dropdown.select_by_visible_text(keyword1)

# 물건종류 선택
time.sleep(1)
ctgr_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr"))))
ctgr_dropdown.select_by_visible_text(keyword2)

time.sleep(1)
# 최대가격 선택
minbAmtEnd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd"))))
minbAmtEnd_dropdown.select_by_visible_text('4억')

지역을 입력해주세요서울


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00C88073+60707]
	GetHandleVerifier [0x00C880B4+60772]
	(No symbol) [0x00AB0683]
	(No symbol) [0x00AF8660]
	(No symbol) [0x00AF89FB]
	(No symbol) [0x00B41022]
	(No symbol) [0x00B1D094]
	(No symbol) [0x00B3E824]
	(No symbol) [0x00B1CE46]
	(No symbol) [0x00AEC5D3]
	(No symbol) [0x00AED424]
	GetHandleVerifier [0x00ECBB53+2435075]
	GetHandleVerifier [0x00EC70F3+2416035]
	GetHandleVerifier [0x00EE349C+2531660]
	GetHandleVerifier [0x00C9F145+155125]
	GetHandleVerifier [0x00CA5AED+182173]
	GetHandleVerifier [0x00C8F948+91640]
	GetHandleVerifier [0x00C8FAF0+92064]
	GetHandleVerifier [0x00C7A5B0+4704]
	BaseThreadInitThunk [0x76FD5D49+25]
	RtlInitializeExceptionChain [0x77BFCF0B+107]
	RtlGetAppContainerNamedObjectPath [0x77BFCE91+561]


In [101]:
driver.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1].text

'아파트\n2024-52691\n서울 강동구 길동 387-5, 7층709호 (길동,길동청광플러스원큐브1차) 외 1필지\n(서울 강동구 천중로42길 70)\n건물 16.685㎡(5.047평), 대지권 4.914㎡(1.486평)\n토지·건물 일괄매각  임차권등기,대항력 있는 임차인,공시가 1억이하'

In [15]:
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

# 물건종류 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr"))))
siCd_dropdown.select_by_visible_text(keyword2)


주거용과 아파트 중 하나를 입력해주세요 아파트


In [18]:
# 최대가격 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd"))))
siCd_dropdown.select_by_visible_text('4억')

In [52]:
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")
# 시작일자 선택
today = datetime.today()
gnDt_datepicker = wait.until(EC.element_to_be_clickable((By.ID, "bgnDt")))
gnDt_datepicker.clear()
gnDt_datepicker.send_keys(start_str)

# 끝일자 선택
endDt_datepicker = wait.until(EC.element_to_be_clickable((By.ID, "endDt")))
endDt_datepicker.clear()
endDt_datepicker.send_keys(end_str)

# 건물면적 최대선택
bldgSqmEnd_input = wait.until(EC.element_to_be_clickable((By.ID, "bldgSqmEnd")))
bldgSqmEnd_input.clear()
bldgSqmEnd_input.send_keys(85)

# 검색버튼
time.sleep(3)
btnSrch = wait.until(EC.element_to_be_clickable((By.ID, "btnSrch")))
btnSrch.click()

# 리스트 갯수 늘리기
dataSize_s_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s"))))
dataSize_s_dropdown.select_by_visible_text('100')

In [ ]:
# 끝일자 선택
endDt_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "endDt"))))
endDt_dropdown.select_by_visible_text(end_str)

In [ ]:
# 최대가격 선택
siCd_dropdown = Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd"))))
siCd_dropdown.click()
siCd_dropdown.select_by_visible_text('4억')

In [49]:
# 검색버튼
time.sleep(2)
btnSrch = wait.until(EC.element_to_be_clickable((By.ID, "btnSrch")))
btnSrch.click()

In [35]:
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

In [51]:
result1 = []
for item_list in driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand'):
    # 사건번호
    num = item_list.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
    # 주소
    addr_div = item_list.find_element(By.CSS_SELECTOR, 'div[id^="adrs_"]')
    address = addr_div.get_attribute("innerText")
    # 감정가
    start_price = int(item_list.find_element(By.ID, "apslAmt_2239070").text.replace(",",""))
    #최저가
    end_price = int(item_list.find_element(By.ID, "minbAmt_2239070").text.replace(",",""))
    # 입찰일
    the_date = item_list.find_element(By.ID, "bidDt_2239070").text
    # 조회수
    pop = item_list.find_element(By.XPATH, '//tr[@id="Tr_1"]//div[@title="[관] : 관심수"]/preceding-sibling::div[1]').text
    print(num,address,start_price,end_price,the_date,pop)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="apslAmt_2239070"]"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100c126b8 chromedriver + 6149816
1   chromedriver                        0x0000000100c09cda chromedriver + 6114522
2   chromedriver                        0x0000000100696d5c chromedriver + 400732
3   chromedriver                        0x00000001006e3aa5 chromedriver + 715429
4   chromedriver                        0x00000001006e3bf1 chromedriver + 715761
5   chromedriver                        0x00000001006d8106 chromedriver + 667910
6   chromedriver                        0x0000000100706cdd chromedriver + 859357
7   chromedriver                        0x00000001006d7fe8 chromedriver + 667624
8   chromedrive

In [83]:

main_window = driver.current_window_handle

# 리스트버튼
time.sleep(2)
clickable = driver.find_element(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')
clickable.click()

# 새로운 창으로 전환
WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
new_window = [window for window in driver.window_handles if window != main_window][0]
driver.switch_to.window(new_window)

# 데이터 추출


# 팝업창 닫기
driver.close()

# 다시 원래 창으로 돌아가기
driver.switch_to.window(main_window)


In [87]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
        )

driver.get('https://www.tankauction.com/ca/caView.php?tid=2273798&chkNo=1&TotNo=49')

wait = WebDriverWait(driver, 10)

In [ ]:
driver.find_element(By.CSS_SELECTOR, '#hitInfo')

In [86]:
driver.find_element(By.CSS_SELECTOR, '#hitInfo')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#hitInfo"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010f6726b8 chromedriver + 6149816
1   chromedriver                        0x000000010f669cda chromedriver + 6114522
2   chromedriver                        0x000000010f0f6d5c chromedriver + 400732
3   chromedriver                        0x000000010f143aa5 chromedriver + 715429
4   chromedriver                        0x000000010f143bf1 chromedriver + 715761
5   chromedriver                        0x000000010f188dd4 chromedriver + 998868
6   chromedriver                        0x000000010f166cdd chromedriver + 859357
7   chromedriver                        0x000000010f1860db chromedriver + 987355
8   chromedriver             

In [ ]:
driver.switch_to.window(driver.window_handles[-1])  

In [79]:
result1 = []
for item_list in driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand'):
    try:
        # 고유 물건 ID 추출 (예: 2239070)
        tid = item_list.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
        
        # 사건번호
        num = item_list.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text

        # 주소
        addr_div = item_list.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}')
        address = addr_div.get_attribute("innerText")

        # 감정가
        start_price = int(item_list.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))

        # 최저가
        end_price = int(item_list.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))

        # 입찰일
        the_date = item_list.find_element(By.ID, f"bidDt_{tid}").text

        # 결과 저장
        result1.append([num, address, start_price, end_price, the_date, pop])


        

        # 출력
        print(num, address, start_price, end_price, the_date)

    except Exception as e:
        print("❗ 오류 발생:", e)

2024-52691 서울 강동구 길동 387-5, 7층709호 (길동,길동청광플러스원큐브1차) 외 1필지 120000000 120000000 25.04.28 
2024-54246 서울 송파구 가락동 78-1, 15층1507호 (가락동,한화오벨리스크) 189000000 189000000 25.04.28 
2023-4493(2) 서울 동대문구 이문동 327-7, 14층1412호 (이문동,세진빌딩) 외 6필지 329000000 263200000 25.04.29 
2023-125267 서울 영등포구 영등포동1가 145, 제9층 제913호 134000000 43909000 25.04.29 
2024-3282 서울 도봉구 창동 811, 101동 2층205호 (도봉동,대동아파트) 485000000 388000000 25.04.29 
2024-106495 서울 도봉구 도봉동 641, 114동 11층1105호 (도봉동,서원아파트) 외 1필지 399000000 319200000 25.04.29 
2024-109326 서울 동대문구 전농동 6, 16동 1층117호 (전농동,전농우성아파트) 외 1필지 536000000 343040000 25.04.29 
2024-111985 서울 도봉구 방학동 272, 16동 12층1210호 (방학동,신동아아파트) 374000000 299200000 25.04.29 
2024-112223 서울 노원구 상계동 639, 103동 12층1213호 (상계동,보람아파트) 외 1필지 411000000 328800000 25.04.29 
2024-114462 서울 강서구 방화동 832, 제303동 제4층 제408호 (방화동, 방화아파트) 189000000 151200000 25.04.29 
2024-142900 서울 강서구 화곡동 938-23, 제8층 제802호 (화곡동, 에스엠케이아파트) 294000000 294000000 25.04.30 
2023-107195 서울 관악구 신림동 1421-1, 10층1004호 (신림동,신림큐브) 140000000 15032

In [106]:
driver.find_elements(By.CSS_SELECTOR, ".hand")[1].get_attribute("innerText")

''

In [44]:
driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand')[60].find_element(By.ID, "adrs_2239070").get_attribute("innerText")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="adrs_2239070"]"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010b8526b8 chromedriver + 6149816
1   chromedriver                        0x000000010b849cda chromedriver + 6114522
2   chromedriver                        0x000000010b2d6d5c chromedriver + 400732
3   chromedriver                        0x000000010b323aa5 chromedriver + 715429
4   chromedriver                        0x000000010b323bf1 chromedriver + 715761
5   chromedriver                        0x000000010b318106 chromedriver + 667910
6   chromedriver                        0x000000010b346cdd chromedriver + 859357
7   chromedriver                        0x000000010b317fe8 chromedriver + 667624
8   chromedriver  

In [42]:
driver.find_element(By.CSS_SELECTOR, ".right")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".clear.f15.bold"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010b8526b8 chromedriver + 6149816
1   chromedriver                        0x000000010b849cda chromedriver + 6114522
2   chromedriver                        0x000000010b2d6d5c chromedriver + 400732
3   chromedriver                        0x000000010b323aa5 chromedriver + 715429
4   chromedriver                        0x000000010b323bf1 chromedriver + 715761
5   chromedriver                        0x000000010b318106 chromedriver + 667910
6   chromedriver                        0x000000010b346cdd chromedriver + 859357
7   chromedriver                        0x000000010b317fe8 chromedriver + 667624
8   chromedriver      

In [83]:
int(driver.find_element(By.ID, "apslAmt_2239070").text.replace(",",""))

372000000

In [56]:
driver.find_element(By.XPATH, '//tr[@id="Tr_1"]//div[@title="[관] : 관심수"]/preceding-sibling::div[1]').text

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=125.0.6422.61)
Stacktrace:
0   chromedriver                        0x0000000100c126b8 chromedriver + 6149816
1   chromedriver                        0x0000000100c09cda chromedriver + 6114522
2   chromedriver                        0x0000000100696d5c chromedriver + 400732
3   chromedriver                        0x000000010067c646 chromedriver + 292422
4   chromedriver                        0x000000010067c533 chromedriver + 292147
5   chromedriver                        0x0000000100699172 chromedriver + 409970
6   chromedriver                        0x0000000100725abb chromedriver + 985787
7   chromedriver                        0x0000000100706a53 chromedriver + 858707
8   chromedriver                        0x00000001006d66d5 chromedriver + 661205
9   chromedriver                        0x00000001006d6f6e chromedr

In [62]:
driver.find_element(By.ID,"saNo_2273798").text

'2024-52691'

In [ ]:

item_list in driver.find_elements(By.CSS_SELECTOR, '#lsTbody > tr.hand'):
# 사건번호
num = item_list.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
# 주소
addr_div = item_list.find_element(By.CSS_SELECTOR, 'div[id^="adrs_"]')
address = addr_div.get_attribute("innerText")
# 감정가
start_price = int(item_list.find_element(By.ID, "apslAmt_2239070").text.replace(",",""))
#최저가
end_price = int(item_list.find_element(By.ID, "minbAmt_2239070").text.replace(",",""))
# 입찰일
the_date = item_list.find_element(By.ID, "bidDt_2239070").text
# 조회수
pop = item_list.find_element(By.XPATH, '//tr[@id="Tr_1"]//div[@title="[관] : 관심수"]/preceding-sibling::div[1]').text
print(num,adress,start_price,end_price,the_date,pop)

In [63]:
driver.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text

'2024-52691'

In [74]:
driver.find_element(By.ID, "apslAmt_2273798").text

'120,000,000'

# 나중에 셀렉트 실행해볼것

In [ ]:
## Ex 5-5. QComboBox.

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QComboBox


class MyApp(QWidget):

    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.lbl = QLabel('Option1', self)
        self.lbl.move(50, 150)

        cb = QComboBox(self)
        cb.addItem('Option1')
        cb.addItem('Option2')
        cb.addItem('Option3')
        cb.addItem('Option4')
        cb.move(50, 50)

        cb.activated[str].connect(self.onActivated)

        self.setWindowTitle('QComboBox')
        self.setGeometry(300, 300, 300, 200)
        self.show()

    def onActivated(self, text):
        self.lbl.setText(text)
        self.lbl.adjustSize()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())